# Trees

In [122]:
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import netCDF4 as n
%matplotlib inline
from utils import train_test_split
from sklearn import tree

## Read the data

Working with netcdf files.

http://unidata.github.io/netcdf4-python/

### Norway

We are not going to use these data (we use files for Europe).

In [2]:
path = "./files/"
filenames = ["specific_humidity_Norway.nc", "relative_humidity_Norway.nc", "pressure_Norway.nc",  
             "temperature_Norway.nc", "total_cloud_cover_Norway.nc"]


cloud = n.Dataset(path + filenames[-1], "r")
relative = n.Dataset(path + filenames[1], "r")
specific = n.Dataset(path + filenames[0], "r")
pressure = n.Dataset(path + filenames[2], "r")
temperature = n.Dataset(path + filenames[3], "r")

files_nor = {'cloud':cloud, 'relative':relative, 'specific':specific, 'pressure':pressure, 'temperature':temperature}

In [3]:
#print(pressure.variables)
tcc_nor = cloud.variables["tcc"]
rh_nor = relative.variables["r"]
# level = relative.variables["level"]
q_nor = specific.variables["q"]
sp_nor = pressure.variables["sp"]
t2m_nor = temperature.variables["t2m"]

### Europe
We use only the first time stamp, and level 1000 hPa.

#### Open files

In [4]:
path = "./files/"
filenames = ["specific_humidity_Europa_sp.nc", "relative_humidity_Europa_sp.nc", "pressure_Europa_sp.nc",  
             "temperature_Europa_sp.nc", "total_cloud_cover_Europa_sp.nc"]


cloud = n.Dataset(path + filenames[-1], "r")
relative = n.Dataset(path + filenames[1], "r")
specific = n.Dataset(path + filenames[0], "r")
pressure = n.Dataset(path + filenames[2], "r")
temperature = n.Dataset(path + filenames[3], "r")

my_files = {'cloud':cloud, 'relative':relative, 'specific':specific, 'pressure':pressure, 'temperature':temperature}

#### Variables

In [5]:
for key, value in my_files.items():
    print(key, ': ', value.variables)

cloud :  OrderedDict([('longitude', <class 'netCDF4._netCDF4.Variable'>
float32 longitude(longitude)
    units: degrees_east
    long_name: longitude
unlimited dimensions: 
current shape = (77,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('latitude', <class 'netCDF4._netCDF4.Variable'>
float32 latitude(latitude)
    units: degrees_north
    long_name: latitude
unlimited dimensions: 
current shape = (61,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('time', <class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:0.0
    long_name: time
    calendar: gregorian
unlimited dimensions: time
current shape = (32,)
filling on, default _FillValue of -2147483647 used
), ('tcc', <class 'netCDF4._netCDF4.Variable'>
int16 tcc(time, latitude, longitude)
    scale_factor: 1.525948758640685e-05
    add_offset: 0.49999237025720666
    _FillValue: -32767
    missing_value: -32767
    units: (0 - 1)
    long_name: Total cloud co

In [6]:
tcc = cloud.variables["tcc"]
rh = relative.variables["r"]
q = specific.variables["q"]
sp = pressure.variables["sp"]
t2m = temperature.variables["t2m"]

my_vars = {'tcc':tcc, 'rh':rh, 'q':q, 'sp':sp, 't2m':t2m}

#### Dimensions and sizes?

In [7]:
for key, value in my_files.items():
    print(key, ': ', value.dimensions)

cloud :  OrderedDict([('longitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 77
), ('latitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 61
), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 32
)])
relative :  OrderedDict([('longitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 77
), ('latitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 61
), ('level', <class 'netCDF4._netCDF4.Dimension'>: name = 'level', size = 6
), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 32
)])
specific :  OrderedDict([('longitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 77
), ('latitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 61
), ('level', <class 'netCDF4._netCDF4.Dimension'>: name = 'level', size = 6
), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 32
)])
pressure

In [8]:
for key, value in my_vars.items():
    print(key, ': ', value.dimensions)

tcc :  ('time', 'latitude', 'longitude')
rh :  ('time', 'level', 'latitude', 'longitude')
q :  ('time', 'level', 'latitude', 'longitude')
sp :  ('time', 'latitude', 'longitude')
t2m :  ('time', 'latitude', 'longitude')


#### Levels

In [9]:
print('rh levels: ', relative['level'][:])
print('q levels: ', specific['level'][:])

rh levels:  [1000  850  700  500  400  300]
q levels:  [1000  850  700  500  400  300]


In [10]:
print('Units for the latitude:', relative.variables['latitude'].units)
print('Units for the longitude:', relative.variables['longitude'].units)
print('Units for the time:', relative.variables['time'].units)
print('Units for the level:', relative.variables['level'].units)

Units for the latitude: degrees_north
Units for the longitude: degrees_east
Units for the time: hours since 1900-01-01 00:00:0.0
Units for the level: millibars


#### Check whether data is corrected for offset and scale_factor

In [11]:
for key, value in my_vars.items():
    test = value.scale
    print(test)
    

True
True
True
True
True


## Write array with data for 1000 hPa, first time stamp

In [57]:
print(tcc.shape)
print(rh.shape)
print(q.shape)
print(sp.shape)
print(t2m.shape)


(32, 61, 77)
(32, 6, 61, 77)
(32, 6, 61, 77)
(32, 61, 77)
(32, 61, 77)


In [118]:
X = np.empty((tcc.shape[1]*tcc.shape[2], 4))
y = np.empty((tcc.shape[1]*tcc.shape[2], 1))

i=0
for key, value in my_vars.items():
    if key == 'tcc':
        y = np.array(value[0, : , :]).flatten()
    elif key == 'rh' or key == 'q':
        X[:, i] = np.array(value[0, 0, : , :]).flatten()
        i = i+1
    else:
        X[:, i] = np.array(value[0, : , :]).flatten()
        i=i+1

#my_vars['tcc'] = tcc[1, :, :]
#my_vars['rh'] = rh[1, 1, :, :]
#my_vars['q'] = q[1, 1, :, :]
#my_vars['sp'] = sp[1, :, :]
#my_vars['t2m'] = t2m[1, :, :]

In [117]:
X[0:10,:]

array([[9.67092498e+01, 7.89155626e-04, 1.00881369e+05, 2.54531033e+02],
       [9.58529196e+01, 8.08777750e-04, 1.00877530e+05, 2.55042680e+02],
       [9.56865236e+01, 8.45675872e-04, 1.00848736e+05, 2.56260888e+02],
       [9.60416369e+01, 8.82787279e-04, 1.00819942e+05, 2.57480033e+02],
       [9.64819773e+01, 9.38027821e-04, 1.00793451e+05, 2.59043087e+02],
       [9.65509708e+01, 1.04573621e-03, 1.00786925e+05, 2.60489006e+02],
       [9.59746727e+01, 1.15387118e-03, 1.00780014e+05, 2.61934925e+02],
       [9.47733753e+01, 1.27160391e-03, 1.00769264e+05, 2.63318059e+02],
       [9.32210230e+01, 1.40192040e-03, 1.00756211e+05, 2.64518462e+02],
       [9.16909922e+01, 1.53074391e-03, 1.00743926e+05, 2.65704809e+02]])

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X, y, split_size=0.5)

In [125]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_train, y_train)

In [126]:
clf.predict(X_test)

array([9.99993896e-01, 9.99993896e-01, 9.98351975e-01, ...,
       9.98046786e-01, 9.99866856e-13, 9.99866856e-13])